In [ ]:
#Accessibility Indicators 2023 
import os
import pandas as pd

root_dir = "AI23/records"

timeColumns = [
    "geo_code",  
    "access_{amenity}_15",
    "access_{amenity}_30",
    "access_{amenity}_45",
    "access_{amenity}_60"
]
nearestColumns=[
    "geo_code",
    "nearest_{amenity}"
]
for amenity in os.listdir(root_dir):
    amenity_path = os.path.join(root_dir, amenity)
    if not os.path.isdir(amenity_path):
        continue
    
    for modeFile in os.listdir(amenity_path):
    
        if modeFile.endswith(".csv"):  
            file_path = os.path.join(amenity_path, modeFile)

            if "pt" in modeFile:
                 df= df[:41729]
                 print(f"cut: {file_path}")
        
            df = pd.read_csv(file_path)
            if amenity == "parks" or amenity=="main_bua":
                columnsKeep=[col.format(amenity=amenity) for col in nearestColumns]
                df_filtered= df[columnsKeep]
                df_filtered.to_csv(file_path, index=False)
                print(f"Processed and saved: {file_path}")
            else:
                columnsKeep = [col.format(amenity=amenity) for col in timeColumns]
                df_filtered = df[columnsKeep]
                df_filtered.to_csv(file_path, index=False)
                print(f"Processed and saved: {file_path}")



In [ ]:
import pandas as pd
df = pd.read_csv("postcode/postcodeMapping.csv")
print("Postcode mapping df unique LSOAs: ",df["lsoa11cd"].nunique())
AIdf=pd.read_csv("AI23/bicycleAI.csv")
print("AI df LSOAs: ", AIdf.shape[0])


Postcode mapping df unique LSOAs:  32844
AI df LSOAs:  32844


In [ ]:
#Police API street crime Data Fetching example
import requests
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
from police_api import PoliceAPI

latitude = 51.471047
longitude = -0.071222
api = PoliceAPI()

latest_month = api.get_latest_date()
# print(latest_month)
latest_month = datetime.strptime(latest_month, "%Y-%m")
# print(latest_month)
last_6_months_crime = []
print(f"Latest month: {latest_month}")

attributes = ["category","location","outcome_status","id","month"]

# Loop to get data for the past 6 months
for i in range(6):
    month = (latest_month - relativedelta(months=i)).strftime("%Y-%m")
    url = f"https://data.police.uk/api/crimes-street/all-crime?date={month}&lat={latitude}&lng={longitude}"
    print(url)
    
    response = requests.get(url)
    if response.status_code == 200:
        crimes = response.json()
        
        # Filter the attributes for each crime
        for crime in crimes:
            filtered_crime = {key: crime[key] for key in attributes if key in crime}
            last_6_months_crime.append(filtered_crime)
    else:
        print(f"Error fetching data: {response.status_code}")
        break


print(f"Total crimes fetched: {len(last_6_months_crime)}")
if last_6_months_crime:  
    df = pd.DataFrame(last_6_months_crime)
    df.to_csv("all_crimes_last_6_monthsfiltered.csv", index=False)
    print("Crime data saved to 'all_crimes_last_6_months.csv'")
else:
    print("No crimes data fetched.")


2024-12
2024-12-01 00:00:00
Latest month: 2024-12-01 00:00:00
https://data.police.uk/api/crimes-street/all-crime?date=2024-12&lat=51.471047&lng=-0.071222
https://data.police.uk/api/crimes-street/all-crime?date=2024-11&lat=51.471047&lng=-0.071222
https://data.police.uk/api/crimes-street/all-crime?date=2024-10&lat=51.471047&lng=-0.071222
https://data.police.uk/api/crimes-street/all-crime?date=2024-09&lat=51.471047&lng=-0.071222
https://data.police.uk/api/crimes-street/all-crime?date=2024-08&lat=51.471047&lng=-0.071222
https://data.police.uk/api/crimes-street/all-crime?date=2024-07&lat=51.471047&lng=-0.071222
Total crimes fetched: 7589
Crime data saved to 'all_crimes_last_6_months.csv'


In [ ]:
category_counts = df["category"].value_counts().reset_index()
category_counts.head()
total_crimes = pd.DataFrame([{"category": "Total", "count": df.shape[0]}])

# Combine the counts and the total row
category_counts = pd.concat([category_counts, total_crimes], ignore_index=True)
print(category_counts.head(20))
df["outcome_category"] = df["outcome_status"].apply(lambda x: x["category"] if isinstance(x, dict) else "Unknown")
outcome_counts = df['outcome_category'].value_counts().reset_index()
outcome_counts.head(20)

                 category  count
0           violent-crime   1649
1   anti-social-behaviour   1446
2             shoplifting   1065
3             other-theft    640
4   theft-from-the-person    537
5           vehicle-crime    490
6   criminal-damage-arson    374
7            public-order    356
8                burglary    332
9                   drugs    223
10                robbery    216
11          bicycle-theft    174
12            other-crime     57
13  possession-of-weapons     30
14                  Total   7589


,outcome_category,count
0,Investigation complete; no suspect identified,3430
1,Unknown,1446
2,Under investigation,1280
3,Unable to prosecute suspect,996
4,Status update unavailable,202
5,Awaiting court outcome,123
6,Local resolution,62
7,Action to be taken by another organisation,24
8,Offender given penalty notice,9
9,Formal action is not in the public interest,7


In [ ]:
#School Ofsted Data Processing

for file in os.listdir(root_dir):
    file_path = os.path.join(root_dir, file)
    if file_path.endswith(".csv"):
        df= pd.read_csv(file_path)
        convert = ['Number of schools inspected', 'Outstanding', 'Good', 'Requires improvement', 'Inadequate']
        df[convert] = df[convert].astype(str)
        df[convert] = df[convert].apply(lambda col: col.str.replace(',', '').astype(int))
        print(df.dtypes)
        df.to_csv(file_path, index=False)
        


Region/Local authority         object
Number of schools inspected     int64
Outstanding                     int64
Good                            int64
Requires improvement            int64
Inadequate                      int64
dtype: object
Region/Local authority         object
Number of schools inspected     int64
Outstanding                     int64
Good                            int64
Requires improvement            int64
Inadequate                      int64
dtype: object
Region/Local authority         object
Number of schools inspected     int64
Outstanding                     int64
Good                            int64
Requires improvement            int64
Inadequate                      int64
dtype: object
Region/Local authority         object
Number of schools inspected     int64
Outstanding                     int64
Good                            int64
Requires improvement            int64
Inadequate                      int64
dtype: object
